# <font color=Green> Lecture 5: Preprocessing  the dataset </font>

## Learning Objectives:
### Students will be able to :
* Deal with missing values in the dataset
    * Count and list records (rows) with missing values
    * count and list attributes (columns) with missing values 
    * remove rows/columns with missing values from Pandas Dataframe
* Do Subsetting/Sampling:
    * Select set of data attributes (column/columns) from the Pandas dataframe 
    * Select set of data records (row/rows) from the Pandas dataframe
* Add/remove rows(records)/columns(attributes) to/from a Pandas dataframe
* Perform Queries on rows/columns of dataframe
    * Select set of rows/columns that satisfy certain conditions or criteria 
    




****

**First we need to import the Pandas package**

In [2]:
import pandas as pd
import numpy as np

# <font color= #2874a6>  Reading the dataset </font>

We will continue using `Stoneflake` dataset we used in week 04

In [3]:
url = "http://archive.ics.uci.edu/ml/machine-learning-databases/00299/StoneFlakes.dat"

mvs = ['?','NA','--','n/a']
notclean = pd.read_csv(url , na_values=mvs , sep='[,\s]+', engine='python')

In [4]:
notclean.shape

(79, 9)

In [5]:
notclean[:5]

,ID,LBI,RTI,WDI,FLA,PSF,FSF,ZDF1,PROZD
0,ar,NaN,35.3,2.60,NaN,42.4,24.2,47.1,69
1,arn,1.23,27.0,3.59,122.0,0.0,40.0,40.0,30
2,be,1.24,26.5,2.90,121.0,16.0,20.7,29.7,72
3,bi1,1.07,29.1,3.10,114.0,44.0,2.6,26.3,68
4,bi2,1.08,43.7,2.40,105.0,32.6,5.8,10.7,42


In [7]:
notclean.columns

Index(['ID', 'LBI', 'RTI', 'WDI', 'FLA', 'PSF', 'FSF', 'ZDF1', 'PROZD'], dtype='object')

# <font color= #2874a6>  Preprocessing  the dataset </font>


## <font color= #af601a> 1 - Dealing with Missing values </font>




 

When the data has missing values, we need to know how bad the data is, and where these missing values are in the data (is it in every row, or is it in a specific column/row that we better drop it), and then decide what to do with missing data (see week 04-part 1 lecture slides for missing data processing techniques). In the follwoing we will discuss some useful commands to investigate and deal with missing values before we start using the data for visualization.  

__Note__: Missing data symbol must be replaced by `nan` before using the following coommands

### <font color= #9a7d0a> 1.1- Identifying and Counting rows/columns with missing values (represented as `NaN`)

#### Use `.isnull()`  or `.isna()` to find `NaN` values anywhere in the  dataframe

`.isnull()` returns a boolean mask which we can use to identify missing values. `.isnull()` is an alias to `.isna`, so basically they are the same. See also `.notna` to do the opposite (i.e. to find not null values) 

In [8]:
# For example to find missing values in the first 5 rows

notclean[:5].isnull()  # note that I'm only using the first 5 rows here!. You need to use all the data set
                       # i.e. notclean[:].isnull()

,ID,LBI,RTI,WDI,FLA,PSF,FSF,ZDF1,PROZD
0,False,True,False,False,True,False,False,False,False
1,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False


#### We can use `.any( axis=0 )` to find columns with `NaN` values

In [12]:
notclean[:].isnull().any(axis=0) 

ID       False
LBI       True
RTI      False
WDI      False
FLA       True
PSF       True
FSF       True
ZDF1      True
PROZD    False
dtype: bool

#### To Find rows that contain `NaN` values use `.any()` with  parameter ``axis=1``

In [11]:
notclean[:].isnull().any(axis=1)
#same as 
notclean[:].isnull().any(axis="columns")

0      True
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8     False
9     False
10    False
11    False
12    False
13    False
14    False
15    False
16    False
17    False
18    False
19    False
20    False
21    False
22    False
23     True
24    False
25    False
26    False
27    False
28    False
29    False
      ...  
49    False
50    False
51    False
52    False
53    False
54    False
55    False
56    False
57    False
58     True
59    False
60    False
61    False
62    False
63    False
64     True
65    False
66    False
67    False
68    False
69    False
70    False
71    False
72    False
73    False
74    False
75    False
76    False
77    False
78    False
Length: 79, dtype: bool

** How  `.any()` works ? **
* If any value in the specified items (iteratable) has a value of **True** it will return **True**
* Using the axis=0 (or axis='rows') to move across rows and check if any of the cells in each colum has the value **True** (i.e. find columns with at least one null), while axis =1 (or axis='columns') will check if any of the cells in each row has the value **True** (i.e., find rows with at least one null). 

So now we can use the mask to show the rows of the dataframe which has missing values (we will discuss indexing Dataframes in detail below)

In [31]:
missingValueRowsMask = notclean[:].isnull().any(axis="columns")
notclean.loc[missingValueRowsMask,:] # list rows with missing values

,ID,LBI,RTI,WDI,FLA,PSF,FSF,ZDF1,PROZD
0,ar,NaN,35.3,2.60,NaN,42.4,24.2,47.1,69
23,ga2,1.22,34.2,2.60,NaN,46.3,0.0,63.6,82
34,l,1.28,19.5,3.44,110.0,0.0,15.0,NaN,98
37,m,1.20,27.2,3.00,121.0,NaN,NaN,25.1,61
58,sk,1.24,34.0,2.70,NaN,NaN,NaN,66.7,85
64,ta,1.22,40.6,2.30,NaN,56.9,13.7,67.8,87


### Let's do some counting
in order to make the right decision about how to preprocess the rows/columns with missing values you may need to run some counting queries about these rows/columns. In the follwoing cells we will write some commands to count number of missing values in each row/column, also counting of number of rows/columns with missing values

#### a-  Counting rows that have  `NaN` values in them

Since `.isnull` and `.any` returns boolean mask, we can sum the returned  boolean mask (True is 1 and False is 0) to get the count of True values in it.

In [13]:
(notclean[:].isnull().any(axis=1) == True).sum()

6

#### b- Counting columns that have  `NaN` values in them

In [14]:
(notclean[:].isnull().any(axis=0) == True).sum()

5

#### c- Listing the columns that have `NaN` values in them, and counting the `NaN` values in each column

In [15]:
notclean.isnull().sum()  # .sum() without axis parameter will use axis =0, and works column-wise

ID       0
LBI      1
RTI      0
WDI      0
FLA      4
PSF      2
FSF      2
ZDF1     1
PROZD    0
dtype: int64

#### d- Counting `NaN` values in the whole dataframe

**i) By counting `Nan`s in each column, then adding the columns counts **

In [16]:
notclean.isnull().sum().sum() # the first sum() count nulls in each column, 
                                # the 2nd sum() sums the column counts

10

or **ii) By flatening the dataframe values, then do the counting.**

* We can get the values of the dataframe as 2D array using the property`.values`
* We can flatten 2D array using `.ravel` 

In [17]:
notclean.isnull().values.ravel().sum()

10

* ravel flatens the array returned by `.isnull().values` before applying `.sum()`
* check `.ravel()` in numpy or Pandas documentation
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.ravel.html

Here is an example of using `.ravel()` to flatten the boolean mask array returned by `.isnull().values` 

In [20]:
v = notclean.isnull().values

In [25]:
# lets see the first 5x5 slice of v
v[0:5,0:5]

array([[False,  True, False, False,  True],
       [False, False, False, False, False],
       [False, False, False, False, False],
       [False, False, False, False, False],
       [False, False, False, False, False]])

In [22]:
#let's see the 1st 6 elements of flattend v
v.ravel()[:6]

array([False,  True, False, False,  True, False])

#### Note: Why do we use `sum()` with masks?

Summing a group of boolean values is similar to counting the `True` values in this group because `True` is treated as `1` and `False` is treated as `0` 

Examples:

In [26]:
False +False

0

In [27]:
True + False

1

In [28]:
True +True +False+True+False

3

In [29]:
a= np.array([True,True ,False,True,False])
a.sum()

3


### <font color= #9a7d0a> 1.2 : removeing rows (records) with missing values

**a)  Using Masks**

Assume we want to keep only rows with no missing values, and remove any row that has one or more missing values.
* We know that the result of this expression  `notclean[:].isnull().any(axis=1)` will be a boolean array with "**False**" for row positions (indexes) that **don't have** any missing (`NaN`) value in them. So We can create a boolean mask using this expression by comparing its result with **False**. That is,
    * The mask will be `notclean[:].isnull().any(axis=1) == False`
        This means, if a row does not have `nan`, the mask will have `False` in the position of that row. If the row has at least one `nan`, then the mask will have `True` in that row position
* Then we can use the mask as index to our dataframe rows to select only the rows that has `True` in their mask index
* Remember that, If `.any()` returned  "**True**" for a row, then that  means the row has at least one attribute with **NaN** value


In [33]:
#Our mask will be
missingValuesRowMask =notclean.isnull().any(axis=1) == False  
# this will return boolean list we can use as Fancy index
missingValuesRowMask[:10]

0    False
1     True
2     True
3     True
4     True
5     True
6     True
7     True
8     True
9     True
dtype: bool

#### Now we use the mask to remove the rows with missing values
Now to remove the rows with missing values we need to use the mask to select only the ones which have no missing values and assign the selection to a new dataframe, say `clean` for example.

But before doing that, let's do some sanity check. Let's check the number of rows before and after removing the bad rows (with missing values)

* **Verification:**
Now if we check the shape of our dataframe before and after cleaning, we should see the difference in the number of rows (the 1st dimension). i.e. the new 1st dimension should be smaller by a number equals to the count of rows with missing value we found above

In [37]:
print(notclean.shape) # check the shape of the notclean data
clean = notclean[notclean.isnull().any(axis=1)==False]  # assign to clean only those rows which any() returned "False"
# or use the mask above
clean = notclean[missingValuesRowMask] # this is the same as previous but in two steps
print(clean.shape) # check the shape of the cleaned data

(79, 9)
(73, 9)


 or use the mask `missingValuesRowMask` above

In [38]:
print(notclean.shape) # check the shape of the notclean data
# assign to clean only those rows which any() returned "False"
clean = notclean[missingValuesRowMask] # this is the same as the previous cell but in two steps
print(clean.shape) # check the shape of the cleaned data

(79, 9)
(73, 9)


Since we have 79 rows in the `nonclean` dataframe, and there was 6 rows with missing values, the `clean` dataframe after removing the rows with missing values has 79-6=73 rows

In [35]:
#count of rows with at least one missing value
(notclean[:].isnull().any(axis=1) == True).sum()

6

Let's look at some rows in the cleaned dataframe

In [36]:
clean[:10]

,ID,LBI,RTI,WDI,FLA,PSF,FSF,ZDF1,PROZD
1,arn,1.23,27.0,3.59,122.0,0.0,40.0,40.0,30
2,be,1.24,26.5,2.90,121.0,16.0,20.7,29.7,72
3,bi1,1.07,29.1,3.10,114.0,44.0,2.6,26.3,68
4,bi2,1.08,43.7,2.40,105.0,32.6,5.8,10.7,42
5,bie,1.39,29.5,2.78,126.0,14.0,0.0,50.0,78
6,bn,1.31,26.3,2.10,119.0,15.7,15.7,30.4,72
7,bo,1.27,27.6,3.50,116.0,16.8,23.0,35.2,69
8,by,1.11,32.6,2.90,113.0,15.8,15.8,15.0,57
9,c,1.32,29.5,2.57,121.0,22.0,2.0,18.0,63
10,cl,1.16,33.4,2.30,131.0,7.5,14.9,6.0,60


#### Another way to remove the rows with missing value(s)  using  mask
###### use `.notnull( )`  and `.all( )`  to select only rows with all values not null (i.e., rows without  `NaN` values)

* In this case we get a boolean mask by asking what are the rows that do not have any null
* `.all()` will return *True* only when all the values in the specified axis are *True* 

In [39]:
# Our mask for all rows that do not have `NaN` (the clean/complete rows)
notclean.notnull().all(axis=1)

0     False
1      True
2      True
3      True
4      True
5      True
6      True
7      True
8      True
9      True
10     True
11     True
12     True
13     True
14     True
15     True
16     True
17     True
18     True
19     True
20     True
21     True
22     True
23    False
24     True
25     True
26     True
27     True
28     True
29     True
      ...  
49     True
50     True
51     True
52     True
53     True
54     True
55     True
56     True
57     True
58    False
59     True
60     True
61     True
62     True
63     True
64    False
65     True
66     True
67     True
68     True
69     True
70     True
71     True
72     True
73     True
74     True
75     True
76     True
77     True
78     True
Length: 79, dtype: bool

In [40]:
print(notclean.shape)
clean=notclean[notclean.notnull().all(axis=1)]
print(clean.shape)

(79, 9)
(73, 9)


In [41]:
clean[:10]

,ID,LBI,RTI,WDI,FLA,PSF,FSF,ZDF1,PROZD
1,arn,1.23,27.0,3.59,122.0,0.0,40.0,40.0,30
2,be,1.24,26.5,2.90,121.0,16.0,20.7,29.7,72
3,bi1,1.07,29.1,3.10,114.0,44.0,2.6,26.3,68
4,bi2,1.08,43.7,2.40,105.0,32.6,5.8,10.7,42
5,bie,1.39,29.5,2.78,126.0,14.0,0.0,50.0,78
6,bn,1.31,26.3,2.10,119.0,15.7,15.7,30.4,72
7,bo,1.27,27.6,3.50,116.0,16.8,23.0,35.2,69
8,by,1.11,32.6,2.90,113.0,15.8,15.8,15.0,57
9,c,1.32,29.5,2.57,121.0,22.0,2.0,18.0,63
10,cl,1.16,33.4,2.30,131.0,7.5,14.9,6.0,60


** b) using Pandas's  `.dropna( )` function **

Removing rows with missing values is easy with pandas, just use `.dropna()` method

In [42]:
print(notclean.shape)
clean=notclean.dropna()
print(clean.shape)


(79, 9)
(73, 9)


See the `.dropna()` help (documentations) for the parameters that control its behavior 

### <font color= #9a7d0a> 1.3-Removing columns (attributes) with missing values
Alternatively, you can drop missing values along the other axis; `axis=1` (or `axis='columns'`). `.dropna(axis='columns')` drops all columns containing a `NaN` value.
* This is not recomended unless the column is useless and has so many missing values
* We can drop columns using masks too. This will be illustrated using indexing techniques below

Example: Lets create a clean version of our `notclean` dataframe by dropping columns with missing values, and save it as `cclean`

In [43]:
print(notclean.shape)
cclean=notclean.dropna(axis='columns')
print(cclean.shape)



(79, 9)
(79, 4)


In [45]:
#counting number of columns with missing values
(notclean[:].isnull().any(axis=0) == True).sum()

5


* ** Verification** :
As we expected, the shape of the new cleaned dataframe has decreased in the columns dimension by  count of columns with missing value

In [46]:
# the new dataframe after some coulmns removed
cclean[:5]

,ID,RTI,WDI,PROZD
0,ar,35.3,2.60,69
1,arn,27.0,3.59,30
2,be,26.5,2.90,72
3,bi1,29.1,3.10,68
4,bi2,43.7,2.40,42


## <font color=#af601a> 2- Selecting a column/ columns </font>

You get columns out of a DataFrame the same way you get elements out of a *dictionary*.

Let's start with a quick review of python dictionaries 

**What is dictionary?**

**Dictionary ** is a Python data structure which is indexed by keys. Keys can be of any type.
It is best to think of a dictionary as a set of (key: value) pairs, with the requirement that the keys are unique (within one dictionary). A pair of braces creates an empty dictionary: like `{}`.
The main operations on a dictionary are storing a value with some key and extracting the value given the key. In addition you can add, remove items to/from dictionaries.

Here are some examples:

In [47]:
# information record as dictionary, the attributes as keys, and the information as values
myInfo ={
  "Name": 'John',
  "Age": 20,
  "Hoppy":'Hiking' 
}
print(myInfo)

{'Name': 'John', 'Age': 20, 'Hoppy': 'Hiking'}


Using `dict` keyword to create the same dictionary above

In [48]:
# another way to create the same dictionary above
myInfo = dict([("Name", 'John'),("Age", 20),   ("Hoppy",'Hiking')])
print(myInfo)

{'Name': 'John', 'Age': 20, 'Hoppy': 'Hiking'}


* You can change the value of a specific item by referring to its key name:

In [49]:
# update value
myInfo['Age'] = 27   # Make sure to use the exact same key (case sensitive)

print(myInfo)

{'Name': 'John', 'Age': 27, 'Hoppy': 'Hiking'}


* Adding an item to the dictionary is done by using a new index key and assigning a value to it:

In [50]:
# add new entry (item) to the dictionary
myInfo['address'] = '17 Drury Lane'   # You need to use new key 
print(myInfo)

{'Name': 'John', 'Age': 27, 'Hoppy': 'Hiking', 'address': '17 Drury Lane'}


* You can access the items of a dictionary by referring to its key name, inside square brackets:

In [51]:
#Get the value of the "Name" key
myname = myInfo["Name"]
myname

'John'

In [52]:
#get the value corresponding to a given key
print(myInfo['Name'])

John


In [53]:
#this will create an error, because the key `name`  doesn't exist 
print(myInfo['name'])

KeyError: 'name'

In [54]:
#This will print the keys
for x in myInfo:
    print(x) 

Name
Age
Hoppy
address


* you can use the `.keys()` function to return keys of a dictionary

In [55]:
#get the keys of dictionary
print(myInfo.keys())

dict_keys(['Name', 'Age', 'Hoppy', 'address'])


* you can also use the `.values()` function to return values of a dictionary
or use `.items()` to get both keys and values


In [56]:
print(myInfo.keys())
print(myInfo.values())
print(myInfo.items())

dict_keys(['Name', 'Age', 'Hoppy', 'address'])
dict_values(['John', 27, 'Hiking', '17 Drury Lane'])
dict_items([('Name', 'John'), ('Age', 27), ('Hoppy', 'Hiking'), ('address', '17 Drury Lane')])


In [57]:
# remove a particular item
myAge = myInfo.pop('Age')
myAge

27

In [58]:
myInfo

{'Name': 'John', 'Hoppy': 'Hiking', 'address': '17 Drury Lane'}

In [59]:
#Put it back
myInfo['Age'] = 27

In [60]:
# delete a particular item
del myInfo['Age']  
myInfo

{'Name': 'John', 'Hoppy': 'Hiking', 'address': '17 Drury Lane'}

### <font color =  #3498db>  Now lets use our Dataframe

### <font color= #9a7d0a>  2.1- Selecting column(s) using column name (Label)

#### Selecting a single column using its name (label)
use the column label as indx, for example `clean['FLA']` will list the column `'FLA'` in our `clean` dataframe

In [61]:
# 1st 5 rows of the cloumn FLA
clean['FLA'][:5]

1    122.0
2    121.0
3    114.0
4    105.0
5    126.0
Name: FLA, dtype: float64

#### Using the dot notations:

If the column name does not have spaces or some other special characters in it then you can use the `dot notations` directly with the column name. 
Example: to access the column `FLA` above we can use `clean.FLA`

Note: Column names are case sensitive, so `clean.FLA` is not the same as `clean.Fla` or `clean['Fla']`, the later two with `'Fla'` as column name will not work 

In [199]:
clean.FLA

0     122.0
1     121.0
2     114.0
3     105.0
4     126.0
5     119.0
6     116.0
7     113.0
8     121.0
9     131.0
10    121.0
11    113.0
12    108.0
13    120.0
14    123.0
15    116.0
16    116.0
17    116.0
18    116.0
19    118.0
20    115.0
21    112.0
22    117.0
23    114.0
24    125.0
25    125.0
26    124.0
27    118.0
28    118.0
29    111.0
      ...  
43    115.0
44    121.0
45    121.0
46    115.0
47    112.0
48    115.0
49    112.0
50    116.0
51    118.0
52    117.0
53    125.0
54    114.0
55    112.0
56    113.0
57    115.0
58    112.0
59    107.0
60    116.0
61    107.0
62    123.0
63    118.0
64    116.0
65    127.0
66    107.0
67    119.0
68    118.0
69    121.0
70    120.0
71    125.0
72    125.0
Name: FLA, Length: 73, dtype: float64

#### Selecting multiple columns using list of column names

In [62]:
clean[['FLA','RTI']][:5]

,FLA,RTI
1,122.0,27.0
2,121.0,26.5
3,114.0,29.1
4,105.0,43.7
5,126.0,29.5


In [63]:
selection =['FLA','RTI']
clean[selection][:5]

,FLA,RTI
1,122.0,27.0
2,121.0,26.5
3,114.0,29.1
4,105.0,43.7
5,126.0,29.5


##### Using a list of  the column names (Table header)
we can use a list to index the dataframe columns

In [64]:
#  Create a list of headers from the `.column` attaribute of the dataframe
headers=list(clean.columns)
headers

['ID', 'LBI', 'RTI', 'WDI', 'FLA', 'PSF', 'FSF', 'ZDF1', 'PROZD']

In [65]:
# we can use the headers list to index the dataframe columns
clean[headers][:5]

,ID,LBI,RTI,WDI,FLA,PSF,FSF,ZDF1,PROZD
1,arn,1.23,27.0,3.59,122.0,0.0,40.0,40.0,30
2,be,1.24,26.5,2.90,121.0,16.0,20.7,29.7,72
3,bi1,1.07,29.1,3.10,114.0,44.0,2.6,26.3,68
4,bi2,1.08,43.7,2.40,105.0,32.6,5.8,10.7,42
5,bie,1.39,29.5,2.78,126.0,14.0,0.0,50.0,78


#### Selecting multiple columns using sliced list of names

In [67]:
# Now we can select some columns using slices of the headers list
clean[headers[1:4]][:5]

,LBI,RTI,WDI
1,1.23,27.0,3.59
2,1.24,26.5,2.90
3,1.07,29.1,3.10
4,1.08,43.7,2.40
5,1.39,29.5,2.78


### <font color= #9a7d0a>  2.2-Select columns using slices
* use ``loc[rowsSlice,columnsSlice]`` to index rows/columns using slicing with their **labels**.
* use ``iloc[rowsSlice,columnsSlice]`` to index rows/columns using slicing with integer **locations** of rows/columns.


#### Select columns using their label (`.loc`)

In [68]:
#Select all rows in columns 'LBI' up to 'PSF'
clean.loc[:,'LBI':'PSF'][:6]

,LBI,RTI,WDI,FLA,PSF
1,1.23,27.0,3.59,122.0,0.0
2,1.24,26.5,2.90,121.0,16.0
3,1.07,29.1,3.10,114.0,44.0
4,1.08,43.7,2.40,105.0,32.6
5,1.39,29.5,2.78,126.0,14.0
6,1.31,26.3,2.10,119.0,15.7


In [83]:
# or we can use the headers list slices in the column index of `.loc`
clean.loc[:,headers[1:6]][:5]
# or you can show all rows by removing the last index[:5]
#clean.loc[:,headers[1:6]] #uncomment this line to see all rows

,LBI,RTI,WDI,FLA,PSF
1,1.23,27.0,3.59,122.0,0.0
2,1.24,26.5,2.90,121.0,16.0
3,1.07,29.1,3.10,114.0,44.0
4,1.08,43.7,2.40,105.0,32.6
5,1.39,29.5,2.78,126.0,14.0


In [84]:
# or directly use slice of rows
clean.loc[:5,headers[1:6]]   # but here :5 means rows index labels, which in this case the same as rows index

,LBI,RTI,WDI,FLA,PSF
1,1.23,27.0,3.59,122.0,0.0
2,1.24,26.5,2.90,121.0,16.0
3,1.07,29.1,3.10,114.0,44.0
4,1.08,43.7,2.40,105.0,32.6
5,1.39,29.5,2.78,126.0,14.0


In [73]:
clean[headers[1:6]][:5]

,LBI,RTI,WDI,FLA,PSF
1,1.23,27.0,3.59,122.0,0.0
2,1.24,26.5,2.90,121.0,16.0
3,1.07,29.1,3.10,114.0,44.0
4,1.08,43.7,2.40,105.0,32.6
5,1.39,29.5,2.78,126.0,14.0


#### Select columns using their int Index (`.iloc`)
note: `iloc` uses the absolute index, not the row/column label. Note the difference when we removed some rows, where the row labels may not match their index. for example, in `notclean`  dataframe row index 0 with lable `0` was removed in `clean` dataframe which made row `0` to have label `1`
We can fix the lables using the `.reset_index()` or `.reindex()` commands

In [ ]:
headers

Let's select the two columns at 1 and 4 , i.e. 'LBI' and 'FLA`

In [87]:
clean.iloc[:,[1,4]][:5]  # note that we use a list of integers(see fancy indexing)

,LBI,FLA
1,1.23,122.0
2,1.24,121.0
3,1.07,114.0
4,1.08,105.0
5,1.39,126.0


Let's use a list of individual indexes (fancy indexing)

In [89]:
# Example the columns indexes at [0,1,2,3,4,5,6,7]
idx = [i for i in range(0,8)] # idx = [0,1,2,3,4,5,6,7]
clean.iloc[:,idx][:7]

,ID,LBI,RTI,WDI,FLA,PSF,FSF,ZDF1
1,arn,1.23,27.0,3.59,122.0,0.0,40.0,40.0
2,be,1.24,26.5,2.90,121.0,16.0,20.7,29.7
3,bi1,1.07,29.1,3.10,114.0,44.0,2.6,26.3
4,bi2,1.08,43.7,2.40,105.0,32.6,5.8,10.7
5,bie,1.39,29.5,2.78,126.0,14.0,0.0,50.0
6,bn,1.31,26.3,2.10,119.0,15.7,15.7,30.4
7,bo,1.27,27.6,3.50,116.0,16.8,23.0,35.2


In [88]:
# Example the columns indexes at [0,2,4,6]
idx = [i for i in range(0,8,2)] # idx = [0,2,4,6]
clean.iloc[:,idx][:5]

,ID,RTI,FLA,FSF
1,arn,27.0,122.0,40.0
2,be,26.5,121.0,20.7
3,bi1,29.1,114.0,2.6
4,bi2,43.7,105.0,5.8
5,bie,29.5,126.0,0.0


#### select rows and columns using fancy indexes and slices 

In [77]:
# select rows from index 2 to (5-1), and columns at indexes 1,2,5
clean.iloc[2:5,[1,2,5]] 

,LBI,RTI,PSF
3,1.07,29.1,44.0
4,1.08,43.7,32.6
5,1.39,29.5,14.0


In [92]:
clean.reset_index()

,index,ID,LBI,RTI,WDI,FLA,PSF,FSF,ZDF1,PROZD
0,1,arn,1.23,27.00,3.59,122.0,0.0,40.0,40.0,30
1,2,be,1.24,26.50,2.90,121.0,16.0,20.7,29.7,72
2,3,bi1,1.07,29.10,3.10,114.0,44.0,2.6,26.3,68
3,4,bi2,1.08,43.70,2.40,105.0,32.6,5.8,10.7,42
4,5,bie,1.39,29.50,2.78,126.0,14.0,0.0,50.0,78
5,6,bn,1.31,26.30,2.10,119.0,15.7,15.7,30.4,72
6,7,bo,1.27,27.60,3.50,116.0,16.8,23.0,35.2,69
7,8,by,1.11,32.60,2.90,113.0,15.8,15.8,15.0,57
8,9,c,1.32,29.50,2.57,121.0,22.0,2.0,18.0,63
9,10,cl,1.16,33.40,2.30,131.0,7.5,14.9,6.0,60


In [ ]:
clean

As you cna see `.reset_index` does not work inplace, so either save in a new dataframe or overwrite the original one. Or, use the parameter `inplace=True`. Also, to prevnt adding new column index use the parameter `drop=True`.

In [95]:
clean.reset_index(drop=True, inplace=True)
clean

,ID,LBI,RTI,WDI,FLA,PSF,FSF,ZDF1,PROZD
0,arn,1.23,27.00,3.59,122.0,0.0,40.0,40.0,30
1,be,1.24,26.50,2.90,121.0,16.0,20.7,29.7,72
2,bi1,1.07,29.10,3.10,114.0,44.0,2.6,26.3,68
3,bi2,1.08,43.70,2.40,105.0,32.6,5.8,10.7,42
4,bie,1.39,29.50,2.78,126.0,14.0,0.0,50.0,78
5,bn,1.31,26.30,2.10,119.0,15.7,15.7,30.4,72
6,bo,1.27,27.60,3.50,116.0,16.8,23.0,35.2,69
7,by,1.11,32.60,2.90,113.0,15.8,15.8,15.0,57
8,c,1.32,29.50,2.57,121.0,22.0,2.0,18.0,63
9,cl,1.16,33.40,2.30,131.0,7.5,14.9,6.0,60


## <font color=#af601a> 3- Selecting rows </font>

### <font color= #9a7d0a>  3.1- using integer index of the row
###### This could be done by:
* 1) indexing colums then use iloc to index the rows. i.e *`clean[columns labels].iloc[int row indexes]`*

In [97]:
clean[headers].iloc[1]

ID         be
LBI      1.24
RTI      26.5
WDI       2.9
FLA       121
PSF        16
FSF      20.7
ZDF1     29.7
PROZD      72
Name: 1, dtype: object

In [96]:
clean[headers].iloc[1:6]

,ID,LBI,RTI,WDI,FLA,PSF,FSF,ZDF1,PROZD
1,be,1.24,26.5,2.90,121.0,16.0,20.7,29.7,72
2,bi1,1.07,29.1,3.10,114.0,44.0,2.6,26.3,68
3,bi2,1.08,43.7,2.40,105.0,32.6,5.8,10.7,42
4,bie,1.39,29.5,2.78,126.0,14.0,0.0,50.0,78
5,bn,1.31,26.3,2.10,119.0,15.7,15.7,30.4,72


In [98]:
clean[:].iloc[[1,5,3]]  # similar to  clean.iloc[[1,5,3],:]

,ID,LBI,RTI,WDI,FLA,PSF,FSF,ZDF1,PROZD
1,be,1.24,26.5,2.9,121.0,16.0,20.7,29.7,72
5,bn,1.31,26.3,2.1,119.0,15.7,15.7,30.4,72
3,bi2,1.08,43.7,2.4,105.0,32.6,5.8,10.7,42


#### OR
* 2) Directly use `iloc` by providing int indexes of the rows and  columns, i.e `.iloc[row,col]` 

In [99]:

clean.iloc[[1,5,3],:]

,ID,LBI,RTI,WDI,FLA,PSF,FSF,ZDF1,PROZD
1,be,1.24,26.5,2.9,121.0,16.0,20.7,29.7,72
5,bn,1.31,26.3,2.1,119.0,15.7,15.7,30.4,72
3,bi2,1.08,43.7,2.4,105.0,32.6,5.8,10.7,42


In [100]:
c = range(5)
clean.iloc[[1,5,3],c]

,ID,LBI,RTI,WDI,FLA
1,be,1.24,26.5,2.9,121.0
5,bn,1.31,26.3,2.1,119.0
3,bi2,1.08,43.7,2.4,105.0


#### OR
* 3) Mix of 1 & 2, i.e *`clean[columns_labels].iloc[int_row_idx,int_col_idx]`*
    * This command will initially  select the columns using the first index (`column_labels`) then refines the selection by selecting only the columns specified in the `iloc` (i.e. using the indexes in  `int_col_idx`)

In [101]:
hdr = clean.columns  # get the current set of columns
colIndx = [hdr[h] for h in (1,3,4,5)]  # select the labeles of the columns we want
colIndx

['LBI', 'WDI', 'FLA', 'PSF']

In [102]:
clean[colIndx][:5]

,LBI,WDI,FLA,PSF
0,1.23,3.59,122.0,0.0
1,1.24,2.90,121.0,16.0
2,1.07,3.10,114.0,44.0
3,1.08,2.40,105.0,32.6
4,1.39,2.78,126.0,14.0


In [103]:
#if we want to further refine the selection to get only 'newCol6' and 'WDI'  at 0,2 resp.
onlySelctItm = [0,2]  

clean[colIndx].iloc[[1,5,3],onlySelctItm]  # from the selected columns [1,3,4,5] only select the first 2 cols,i.e. select 1,3

,LBI,FLA
1,1.24,121.0
5,1.31,119.0
3,1.08,105.0


In [104]:
clean[colIndx][:5]

,LBI,WDI,FLA,PSF
0,1.23,3.59,122.0,0.0
1,1.24,2.90,121.0,16.0
2,1.07,3.10,114.0,44.0
3,1.08,2.40,105.0,32.6
4,1.39,2.78,126.0,14.0


In [105]:
clean.iloc[:5] # if columns index specified with .iloc[], then select all columns

,ID,LBI,RTI,WDI,FLA,PSF,FSF,ZDF1,PROZD
0,arn,1.23,27.0,3.59,122.0,0.0,40.0,40.0,30
1,be,1.24,26.5,2.90,121.0,16.0,20.7,29.7,72
2,bi1,1.07,29.1,3.10,114.0,44.0,2.6,26.3,68
3,bi2,1.08,43.7,2.40,105.0,32.6,5.8,10.7,42
4,bie,1.39,29.5,2.78,126.0,14.0,0.0,50.0,78


If we only want the values (i.e. no dataframe, no column headers).  Use ``.values``

Useful to store the data in array

In [106]:
data = clean.iloc[:,1:].values  # get values of all rows and columns except the 1st column 
data.shape

(73, 8)

In [107]:
data[:5,:]  # list the 1st 5 rows and all columns

array([[  1.23,  27.  ,   3.59, 122.  ,   0.  ,  40.  ,  40.  ,  30.  ],
       [  1.24,  26.5 ,   2.9 , 121.  ,  16.  ,  20.7 ,  29.7 ,  72.  ],
       [  1.07,  29.1 ,   3.1 , 114.  ,  44.  ,   2.6 ,  26.3 ,  68.  ],
       [  1.08,  43.7 ,   2.4 , 105.  ,  32.6 ,   5.8 ,  10.7 ,  42.  ],
       [  1.39,  29.5 ,   2.78, 126.  ,  14.  ,   0.  ,  50.  ,  78.  ]])

### <font color= #9a7d0a> 3.2 - Using rows labels:
    If the rows has a special index (labels), it is also possible to use them as index

Example: Let's create new dataframe with row labels as rows index, then use it to index the rows

In [108]:
x = pd.DataFrame({'x': [1, 2, 3], 'y': [3, 4, 5]}, index = list('abc'))
x

,x,y
a,1,3
b,2,4
c,3,5


To select rows `a` and `b` and only column `x`

In [112]:
x.loc['a':'b', 'x']

a    1
b    2
Name: x, dtype: int64

Doing the same using their int indexs, i.e. row with label `a` is at index `0`, and row with lable `b` is at index one, i.e. slice `0:2` will select them or simply the list `[0,1]` in rows index. similarlly  column with label `x` is at index `0`  

In [114]:
x.iloc[0:2,0] 
# or 
#x.iloc[ [0,1] , 0 ]

a    1
b    2
Name: x, dtype: int64

In [115]:
x[['x','y']]

,x,y
a,1,3
b,2,4
c,3,5


##  <font color=#af601a> 4- Adding rows using ``append()``

* let's try to add a copy of the 3rd row data, execluding the 1st column, as a new row to our dataframe

In [116]:
# this is the row data we want to add
clean.iloc[2,1:]

LBI      1.07
RTI      29.1
WDI       3.1
FLA       114
PSF        44
FSF       2.6
ZDF1     26.3
PROZD      68
Name: 2, dtype: object

In [117]:
#Adding a copy of the 3rd row
clean.append(clean.iloc[2,1:], ignore_index=True)

,ID,LBI,RTI,WDI,FLA,PSF,FSF,ZDF1,PROZD
0,arn,1.23,27.00,3.59,122.0,0.0,40.0,40.0,30
1,be,1.24,26.50,2.90,121.0,16.0,20.7,29.7,72
2,bi1,1.07,29.10,3.10,114.0,44.0,2.6,26.3,68
3,bi2,1.08,43.70,2.40,105.0,32.6,5.8,10.7,42
4,bie,1.39,29.50,2.78,126.0,14.0,0.0,50.0,78
5,bn,1.31,26.30,2.10,119.0,15.7,15.7,30.4,72
6,bo,1.27,27.60,3.50,116.0,16.8,23.0,35.2,69
7,by,1.11,32.60,2.90,113.0,15.8,15.8,15.0,57
8,c,1.32,29.50,2.57,121.0,22.0,2.0,18.0,63
9,cl,1.16,33.40,2.30,131.0,7.5,14.9,6.0,60


* Let's see if the new row was added successfully 

In [118]:
# the shape was (73,9) before the addition
clean.shape

(73, 9)

##### Note: 
Append doesn't change the dataframe, what?

Yes, it doesn't work in-place, it returns a new dataframe with the added rows.
So you need to save the changes into a new dataframe

In [120]:
# append doesnt change the dataframe
clean[-5:] # the last 5 items

,ID,LBI,RTI,WDI,FLA,PSF,FSF,ZDF1,PROZD
68,wn,1.04,30.4,2.70,118.0,33.3,5.1,28.2,66
69,woe,1.14,31.9,2.07,121.0,15.0,10.0,5.0,47
70,wol,1.24,27.2,3.00,120.0,29.1,13.8,34.4,70
71,wst,1.15,38.0,2.25,125.0,9.5,6.4,13.0,88
72,z,1.37,26.0,3.22,125.0,12.6,21.5,28.6,61


So let's save the changes into a new dataframe `clean2`

In [123]:
#you need to save the changes into a new dataframe
clean2 = clean.append(clean.iloc[1,4:])
clean2[-5:]

,ID,LBI,RTI,WDI,FLA,PSF,FSF,ZDF1,PROZD
69,woe,1.14,31.9,2.07,121.0,15.0,10.0,5.0,47
70,wol,1.24,27.2,3.00,120.0,29.1,13.8,34.4,70
71,wst,1.15,38.0,2.25,125.0,9.5,6.4,13.0,88
72,z,1.37,26.0,3.22,125.0,12.6,21.5,28.6,61
1,NaN,NaN,NaN,NaN,121.0,16.0,20.7,29.7,72


In [124]:
clean2.shape

(74, 9)

using ``ignore_index=True``
* notice the changes in the index values

In [125]:
clean2 = clean.append(clean.iloc[1,1:], ignore_index=True)
clean2[-5:]


,ID,LBI,RTI,WDI,FLA,PSF,FSF,ZDF1,PROZD
69,woe,1.14,31.9,2.07,121.0,15.0,10.0,5.0,47
70,wol,1.24,27.2,3.00,120.0,29.1,13.8,34.4,70
71,wst,1.15,38.0,2.25,125.0,9.5,6.4,13.0,88
72,z,1.37,26.0,3.22,125.0,12.6,21.5,28.6,61
73,NaN,1.24,26.5,2.90,121.0,16.0,20.7,29.7,72


In [126]:
clean2[:5]

,ID,LBI,RTI,WDI,FLA,PSF,FSF,ZDF1,PROZD
0,arn,1.23,27.0,3.59,122.0,0.0,40.0,40.0,30
1,be,1.24,26.5,2.90,121.0,16.0,20.7,29.7,72
2,bi1,1.07,29.1,3.10,114.0,44.0,2.6,26.3,68
3,bi2,1.08,43.7,2.40,105.0,32.6,5.8,10.7,42
4,bie,1.39,29.5,2.78,126.0,14.0,0.0,50.0,78


## <font color=#af601a> 5- Adding Columns to a Dataframe </font>

In [127]:
# To suppress a warning about chained assignments 
pd.options.mode.chained_assignment = None  # default='warn'

### <font color= #9a7d0a> 5.1-  Add column using a new column name

#### 5.1.1 - Simplist way: add the new column using default value ( e.g. 0, or 'default')


In [129]:
clean['newCol'] = 0 # if the column doesnot exist it will be created
clean[:5]


,ID,LBI,RTI,WDI,FLA,PSF,FSF,ZDF1,PROZD,newCol
0,arn,1.23,27.0,3.59,122.0,0.0,40.0,40.0,30,0
1,be,1.24,26.5,2.90,121.0,16.0,20.7,29.7,72,0
2,bi1,1.07,29.1,3.10,114.0,44.0,2.6,26.3,68,0
3,bi2,1.08,43.7,2.40,105.0,32.6,5.8,10.7,42,0
4,bie,1.39,29.5,2.78,126.0,14.0,0.0,50.0,78,0


In [130]:
clean.loc[:,'newCol2'] = -1
clean[:5]

,ID,LBI,RTI,WDI,FLA,PSF,FSF,ZDF1,PROZD,newCol,newCol2
0,arn,1.23,27.0,3.59,122.0,0.0,40.0,40.0,30,0,-1
1,be,1.24,26.5,2.90,121.0,16.0,20.7,29.7,72,0,-1
2,bi1,1.07,29.1,3.10,114.0,44.0,2.6,26.3,68,0,-1
3,bi2,1.08,43.7,2.40,105.0,32.6,5.8,10.7,42,0,-1
4,bie,1.39,29.5,2.78,126.0,14.0,0.0,50.0,78,0,-1


#### 5.1.2-  Using data from other column

In [131]:
clean['newCol3']=clean['FSF']

In [132]:
clean[:5]

,ID,LBI,RTI,WDI,FLA,PSF,FSF,ZDF1,PROZD,newCol,newCol2,newCol3
0,arn,1.23,27.0,3.59,122.0,0.0,40.0,40.0,30,0,-1,40.0
1,be,1.24,26.5,2.90,121.0,16.0,20.7,29.7,72,0,-1,20.7
2,bi1,1.07,29.1,3.10,114.0,44.0,2.6,26.3,68,0,-1,2.6
3,bi2,1.08,43.7,2.40,105.0,32.6,5.8,10.7,42,0,-1,5.8
4,bie,1.39,29.5,2.78,126.0,14.0,0.0,50.0,78,0,-1,0.0


In [133]:
clean['ffff']= clean['WDI'] + clean['FSF']
clean[:5]

,ID,LBI,RTI,WDI,FLA,PSF,FSF,ZDF1,PROZD,newCol,newCol2,newCol3,ffff
0,arn,1.23,27.0,3.59,122.0,0.0,40.0,40.0,30,0,-1,40.0,43.59
1,be,1.24,26.5,2.90,121.0,16.0,20.7,29.7,72,0,-1,20.7,23.60
2,bi1,1.07,29.1,3.10,114.0,44.0,2.6,26.3,68,0,-1,2.6,5.70
3,bi2,1.08,43.7,2.40,105.0,32.6,5.8,10.7,42,0,-1,5.8,8.20
4,bie,1.39,29.5,2.78,126.0,14.0,0.0,50.0,78,0,-1,0.0,2.78


#### 5.1.3- Using Series function

* Series is another Pandas data container. Each coulmn in the dataframe is considered a Series

* First,  let's get #of rows in dataframe so we create a series with the same length

In [134]:
#Dataframe has shape property like numpy arrays
clean.shape

(73, 13)

In [135]:
#get the 1st dimension in variable L
L=clean.shape[0]
L

73

#### OR

In [138]:
L=len(clean['ID']) # or L = len(clean)
L

73

* Now lets create the new column using a series of length of the datafarme

In [139]:
#generate a random series of length L
clean['newCol4'] = pd.Series(np.random.randn(L)) 

In [140]:
clean[:5]

,ID,LBI,RTI,WDI,FLA,PSF,FSF,ZDF1,PROZD,newCol,newCol2,newCol3,ffff,newCol4
0,arn,1.23,27.0,3.59,122.0,0.0,40.0,40.0,30,0,-1,40.0,43.59,-2.211314
1,be,1.24,26.5,2.90,121.0,16.0,20.7,29.7,72,0,-1,20.7,23.60,-0.750064
2,bi1,1.07,29.1,3.10,114.0,44.0,2.6,26.3,68,0,-1,2.6,5.70,-0.343010
3,bi2,1.08,43.7,2.40,105.0,32.6,5.8,10.7,42,0,-1,5.8,8.20,1.718626
4,bie,1.39,29.5,2.78,126.0,14.0,0.0,50.0,78,0,-1,0.0,2.78,0.717959


#### 5.1.4 - Adding column from array/list

In [141]:
# create some numpy array 
arr = np.linspace(0,1,len(clean['ID']))

#add the arrray as new column
clean['newCol5'] = arr  
clean[:5]

,ID,LBI,RTI,WDI,FLA,PSF,FSF,ZDF1,PROZD,newCol,newCol2,newCol3,ffff,newCol4,newCol5
0,arn,1.23,27.0,3.59,122.0,0.0,40.0,40.0,30,0,-1,40.0,43.59,-2.211314,0.000000
1,be,1.24,26.5,2.90,121.0,16.0,20.7,29.7,72,0,-1,20.7,23.60,-0.750064,0.013889
2,bi1,1.07,29.1,3.10,114.0,44.0,2.6,26.3,68,0,-1,2.6,5.70,-0.343010,0.027778
3,bi2,1.08,43.7,2.40,105.0,32.6,5.8,10.7,42,0,-1,5.8,8.20,1.718626,0.041667
4,bie,1.39,29.5,2.78,126.0,14.0,0.0,50.0,78,0,-1,0.0,2.78,0.717959,0.055556


### 5.2-  Add column and specify its location (using ``insert``)

#### NOTE: <font color=red> column shouldn't exist before calling insert, otherwise you'll get an error unless  allow_duplicates=True is used</font>

clean.insert(location, newcolumn, Defaultvalue, allow_duplicates=False)

In [142]:
# this should produce an error
clean.insert(1, 'newCol5', 0)

ValueError: cannot insert newCol5, already exists

In [143]:
#here, we are inserting at index 1 (so should be second col in dataframe)
clean.insert(1, 'newCol6', 'AAA')
clean[:5]


,ID,newCol6,LBI,RTI,WDI,FLA,PSF,FSF,ZDF1,PROZD,newCol,newCol2,newCol3,ffff,newCol4,newCol5
0,arn,AAA,1.23,27.0,3.59,122.0,0.0,40.0,40.0,30,0,-1,40.0,43.59,-2.211314,0.000000
1,be,AAA,1.24,26.5,2.90,121.0,16.0,20.7,29.7,72,0,-1,20.7,23.60,-0.750064,0.013889
2,bi1,AAA,1.07,29.1,3.10,114.0,44.0,2.6,26.3,68,0,-1,2.6,5.70,-0.343010,0.027778
3,bi2,AAA,1.08,43.7,2.40,105.0,32.6,5.8,10.7,42,0,-1,5.8,8.20,1.718626,0.041667
4,bie,AAA,1.39,29.5,2.78,126.0,14.0,0.0,50.0,78,0,-1,0.0,2.78,0.717959,0.055556


## <font color=blue>  6 - Removing columns/rows using </font>``drop()``
#### Note: 
* ``drop`` do not work directly on the datafram, instead it returns a new dataframe object, so you need to save the changes
*  <font color=red> column should exist before calling drop(), otherwise you'll get an error </font>


* we can delete multiple colums using a list of their column names, and using ``axis = 1`` parameter
* drop does not work inplace by default, so you either save the result or use the parameter `inplace=True` to replace the original dataframe with the changed one

In [144]:
clean.drop(['newCol5','newCol2', 'newCol','newCol3'], axis=1)

,ID,newCol6,LBI,RTI,WDI,FLA,PSF,FSF,ZDF1,PROZD,ffff,newCol4
0,arn,AAA,1.23,27.00,3.59,122.0,0.0,40.0,40.0,30,43.59,-2.211314
1,be,AAA,1.24,26.50,2.90,121.0,16.0,20.7,29.7,72,23.60,-0.750064
2,bi1,AAA,1.07,29.10,3.10,114.0,44.0,2.6,26.3,68,5.70,-0.343010
3,bi2,AAA,1.08,43.70,2.40,105.0,32.6,5.8,10.7,42,8.20,1.718626
4,bie,AAA,1.39,29.50,2.78,126.0,14.0,0.0,50.0,78,2.78,0.717959
5,bn,AAA,1.31,26.30,2.10,119.0,15.7,15.7,30.4,72,17.80,-0.037007
6,bo,AAA,1.27,27.60,3.50,116.0,16.8,23.0,35.2,69,26.50,0.487745
7,by,AAA,1.11,32.60,2.90,113.0,15.8,15.8,15.0,57,18.70,-0.879200
8,c,AAA,1.32,29.50,2.57,121.0,22.0,2.0,18.0,63,4.57,-0.971836
9,cl,AAA,1.16,33.40,2.30,131.0,7.5,14.9,6.0,60,17.20,0.586996


In [145]:
#No change to clean
clean[-5:]

,ID,newCol6,LBI,RTI,WDI,FLA,PSF,FSF,ZDF1,PROZD,newCol,newCol2,newCol3,ffff,newCol4,newCol5
68,wn,AAA,1.04,30.4,2.70,118.0,33.3,5.1,28.2,66,0,-1,5.1,7.80,-0.510279,0.944444
69,woe,AAA,1.14,31.9,2.07,121.0,15.0,10.0,5.0,47,0,-1,10.0,12.07,1.322430,0.958333
70,wol,AAA,1.24,27.2,3.00,120.0,29.1,13.8,34.4,70,0,-1,13.8,16.80,0.046687,0.972222
71,wst,AAA,1.15,38.0,2.25,125.0,9.5,6.4,13.0,88,0,-1,6.4,8.65,1.753506,0.986111
72,z,AAA,1.37,26.0,3.22,125.0,12.6,21.5,28.6,61,0,-1,21.5,24.72,-0.361093,1.000000


In [146]:
# to force the change, re assign the result to the smae dataframe
clean =  clean.drop(['newCol5','newCol2', 'newCol','newCol3','newCol4'], axis=1)
clean[-5:]

,ID,newCol6,LBI,RTI,WDI,FLA,PSF,FSF,ZDF1,PROZD,ffff
68,wn,AAA,1.04,30.4,2.70,118.0,33.3,5.1,28.2,66,7.80
69,woe,AAA,1.14,31.9,2.07,121.0,15.0,10.0,5.0,47,12.07
70,wol,AAA,1.24,27.2,3.00,120.0,29.1,13.8,34.4,70,16.80
71,wst,AAA,1.15,38.0,2.25,125.0,9.5,6.4,13.0,88,8.65
72,z,AAA,1.37,26.0,3.22,125.0,12.6,21.5,28.6,61,24.72


In [148]:
clean = clean.drop(['ffff'], axis=1)
clean[:5]

,ID,LBI,RTI,WDI,FLA,PSF,FSF,ZDF1,PROZD
0,arn,1.23,27.0,3.59,122.0,0.0,40.0,40.0,30
1,be,1.24,26.5,2.90,121.0,16.0,20.7,29.7,72
2,bi1,1.07,29.1,3.10,114.0,44.0,2.6,26.3,68
3,bi2,1.08,43.7,2.40,105.0,32.6,5.8,10.7,42
4,bie,1.39,29.5,2.78,126.0,14.0,0.0,50.0,78


In [147]:
# or use inplace=True
clean.drop(['newCol6'], axis=1, inplace=True)
clean[:5]

,ID,LBI,RTI,WDI,FLA,PSF,FSF,ZDF1,PROZD,ffff
0,arn,1.23,27.0,3.59,122.0,0.0,40.0,40.0,30,43.59
1,be,1.24,26.5,2.90,121.0,16.0,20.7,29.7,72,23.60
2,bi1,1.07,29.1,3.10,114.0,44.0,2.6,26.3,68,5.70
3,bi2,1.08,43.7,2.40,105.0,32.6,5.8,10.7,42,8.20
4,bie,1.39,29.5,2.78,126.0,14.0,0.0,50.0,78,2.78


* To delete row, use ``drop()`` with just index or ``axis = 0``

In [149]:
# note index is according to the row index label (index column), here it starts from 1
clean2 = clean.drop([1,2])
clean2[:5]

,ID,LBI,RTI,WDI,FLA,PSF,FSF,ZDF1,PROZD
0,arn,1.23,27.0,3.59,122.0,0.0,40.0,40.0,30
3,bi2,1.08,43.7,2.40,105.0,32.6,5.8,10.7,42
4,bie,1.39,29.5,2.78,126.0,14.0,0.0,50.0,78
5,bn,1.31,26.3,2.10,119.0,15.7,15.7,30.4,72
6,bo,1.27,27.6,3.50,116.0,16.8,23.0,35.2,69


## <font color=blue>  7- Perform Queries on rows/columns of dataframe

### Find all rows that have specific value

* you can look for substring  in columns of type string  using ``in`` clause

In [150]:
# this will return True because the substring 'cd' is in the string 'abcdef'
'cdfe' in 'abcdef'

False

In [152]:
'cde' in  'abcdef'

True

#### Using multiple conditions to create the mask
We can use the logical bitwise oberators to create complex conditions. Python supports the following bitwise operators with boolean arrays:
For any tow boolean arrays x,y we have:
   
__x & y__
    
    Does a "element-wise and". Each element of the output is True if the corresponding element of x AND of y is True, otherwise it's False. 

__x | y__
    
    Does a "element-wise or". Each element of the output is False if the corresponding element of x AND of y is False, otherwise it's True. 

__~ x__
    
    Returns the complement of x - the elements you get by switching each True for a False and each False for a True.  

__x ^ y__
    
    Does a "element-wise exclusive or". Each element of the output is the same as the corresponding element in x if that element in y is False, and it's the complement of the element in x if that element in y is True. 

here we will use `&` (and)  and the `|` (Or) opeartors 

Example

In [231]:
a=np.array([1,2,3,4,5,6])

print ('AND:\n','x=  ',(a>=2), '\n y=  ',(a<=4))
print (' x&y=',(a>=2) & (a<=4))

print ('\n OR:\n','x=  ',(a<=2) , '\n y=  ',(a>=5) )
print (' x|y=',(a<=2) | (a>=5) )

print ('\n Not:\n','x=  ',(a%2==0))
print ('~x=  ',~(a%2==0))
print("\n Other examples")
print ((a%2==0) & (a>2) )


AND:
 x=   [False  True  True  True  True  True] 
 y=   [ True  True  True  True False False]
 x&y= [False  True  True  True False False]

 OR:
 x=   [ True  True False False False False] 
 y=   [False False False False  True  True]
 x|y= [ True  True False False  True  True]

 Not:
 x=   [False  True False  True False  True]
~x=   [ True False  True False  True False]

 Other examples
[False False False  True False  True]


##### Remember: int index  Vs Label index of data frame (`.iloc[ ]` vs `.loc[ ]`)

We can use integer indexes through ``.iloc[]`` or  label indexes through ``.loc[]`` :
* `.iloc[]`  to access dataframe values using only integer (or boolean) indexes of row/column 
* `.loc[]` to access dataframe values using column/row label

### <font color =   #1abc9c  > Example: Let's create code to answer the following queries on our StoneFlake dataset </font>
<font color =   #34495e >
<p>
    <dd> Q1: Select all rows that have the substring *'bi'* in the **ID** column </dd>
    <dd> Q2: Calculate the average of a column (say `ZDF1`) in a set of selected rows(not the   whole column, e.g. the rows selected by Q1)</dd>
    <dd> Q3: List the values less than or equal (&le;) the average (calculated by Q2) in the rows selected by Q1</dd>
    <dd> Q4: Count the frequency (counts) of each value in a column  </dd>
    <dd>Q5: List all unique values in a column</dd>
</p>
</font>

####  <font color =  #FF5733 > Q1: Select all rows that have the substring *'bi'* in the **ID** column

In [ ]:
#here is one way to look at the 1st column using its int index and iloc
clean.iloc[:,0][:8]

In [ ]:
#here is another way to look at the 1st column using its label index and loc
clean.loc[:,'ID'][:8]

step 1: create a mask for the column 'ID' that shows only rows with strings contain the substring 'bi' in this column

In [153]:
# 1st let's create a mask for the column where the strings contain the substring 'bi'

selctedRowsMask = ["bi" in  v for v in clean['ID'] ]
 
# see the fisrt 10 values in our mask
selctedRowsMask[:10] 

[False, False, True, True, True, False, False, False, False, False]

In [154]:
# same as above, but using loc instead of iloc
selctedRowsMask = ["bi" in  v for v in clean.loc[:,'ID'] ]
selctedRowsMask[:10]

[False, False, True, True, True, False, False, False, False, False]

let's see how many rows has substring `bi` in them

In [155]:
sum(selctedRowsMask)

3

Sanity Check: Our mask size must be equal to the # of rows, why?

In [156]:
len(selctedRowsMask)

73

Step 2: To find the index of these rows selected by the mask using `np.where`

In [157]:
Found = np.where(selctedRowsMask)
i = Found[0]
i

array([2, 3, 4], dtype=int64)

In [158]:

# to get an array  as a list use .toList()
#example
Found[0].tolist()


[2, 3, 4]

In [159]:
#convert to list directly 
Found = np.where(selctedRowsMask)
i = Found[0].tolist()
i

[2, 3, 4]

Step 3:  access/list  the selected rows (with 'bi' in the first col) using the locations in `i`

In [160]:
clean.iloc[i,:]

,ID,LBI,RTI,WDI,FLA,PSF,FSF,ZDF1,PROZD
2,bi1,1.07,29.1,3.10,114.0,44.0,2.6,26.3,68
3,bi2,1.08,43.7,2.40,105.0,32.6,5.8,10.7,42
4,bie,1.39,29.5,2.78,126.0,14.0,0.0,50.0,78


We may store the result of this query

In [166]:
Q1Result = clean.iloc[i]
Q1Result

,ID,LBI,RTI,WDI,FLA,PSF,FSF,ZDF1,PROZD
2,bi1,1.07,29.1,3.10,114.0,44.0,2.6,26.3,68
3,bi2,1.08,43.7,2.40,105.0,32.6,5.8,10.7,42
4,bie,1.39,29.5,2.78,126.0,14.0,0.0,50.0,78


####  <font color =  #FF5733 > Q2: Calculate the average of a column in a set of selected rows ( <font color=blue> not the whole column </font>)

for small data such above we can do that manually!

In [162]:

(26.3+10.7+50.0)/3


29.0

Example: calculate average of the column `ZDF1` for the rows in the previous selection (by Q1)

In [163]:
# Example: To calculate average of the column `ZDF1` for the rows in the previous selection
avgSel = clean['ZDF1'].iloc[i].sum()/len(i)
avgSel

29.0

We may use `.sum()` and devide by length of our column (or dataframe)

In [164]:
# Example: To calculate average of the whole column `ZDF1`
avg1 = clean['ZDF1'].sum()/len(clean)
avg1

43.54109589041096

In [165]:
clean['ZDF1'].sum()/73

43.54109589041096

We may use `.mean()` also

In [167]:
clean['ZDF1'].mean()

43.54109589041095

#### <font color =  #FF5733 > Q3: List the rows with values <= the calculated average ( <font color=green> i.e from the rows selected by Q1  select the rows with 'ZDF1' column values <= the avaerage </font>)

step1: create a boolean mask according to the condition "<=avgSel" calculated in Q2

In [168]:
# we will use  Q1 results stored in Q1Result dataframe above
ltAvgSelMask = Q1Result['ZDF1'] <= avgSel
ltAvgSelMask

2     True
3     True
4    False
Name: ZDF1, dtype: bool

or do that using the index `i` created by `np.where` above

In [ ]:
#step1: create a boolean mask according to the condition "<=avgSel"
ltAvgSelMask = clean['ZDF1'].iloc[i] <= avgSel
ltAvgSelMask

#### Then we have 3 choices for step 2

**step 2.Choice#1:** Use `np.where` to find the locations of selected rows in the mask, and then use that to index the results of Q1

In [169]:

found = np.where(ltAvgSelMask)
ii = found[0].tolist()
ii

[0, 1]

In [170]:
Q1Result.iloc[ii]

,ID,LBI,RTI,WDI,FLA,PSF,FSF,ZDF1,PROZD
2,bi1,1.07,29.1,3.1,114.0,44.0,2.6,26.3,68
3,bi2,1.08,43.7,2.4,105.0,32.6,5.8,10.7,42


**Step 2.Choice#2 :** Use the mask `ltAvgSelMask` to select the indexes in the previous index `i` resulted from `Q1`, then use it to index the original dataframe `clean`

In [171]:
ii = Found[0][ltAvgSelMask]  # from Found[0] get only those selected by the mask
ii = ii.tolist() # convert it to list
#  ii = Found[0][ltAvgSelMask].tolist()  # same as above in one step
ii

[2, 3]

In [172]:
ii = np.array(i)[ltAvgSelMask] ##
ii.tolist()

[2, 3]

Get the rows using their location in the original dataframe `clean`

Use `iloc[]` to list the rows

In [173]:
#step3: list the rows using the locations index list
clean.iloc[ii]

,ID,LBI,RTI,WDI,FLA,PSF,FSF,ZDF1,PROZD
2,bi1,1.07,29.1,3.1,114.0,44.0,2.6,26.3,68
3,bi2,1.08,43.7,2.4,105.0,32.6,5.8,10.7,42


**Step 2.Choice#3 :** Using both masks directly

In [174]:
# just use both masks, (Q1) that filtered rows using col 0 containing 'bi', and 
# (Q3) mask that further filtered rows that are <= avg
clean[selctedRowsMask][ltAvgSelMask]

,ID,LBI,RTI,WDI,FLA,PSF,FSF,ZDF1,PROZD
2,bi1,1.07,29.1,3.1,114.0,44.0,2.6,26.3,68
3,bi2,1.08,43.7,2.4,105.0,32.6,5.8,10.7,42


Alternatively 

In [175]:
clean.loc[selctedRowsMask,:].loc[ltAvgSelMask,:]

,ID,LBI,RTI,WDI,FLA,PSF,FSF,ZDF1,PROZD
2,bi1,1.07,29.1,3.1,114.0,44.0,2.6,26.3,68
3,bi2,1.08,43.7,2.4,105.0,32.6,5.8,10.7,42


#### <font color =  #FF5733 > Q4: Count the frequency (counts) of each value in a column using ``value_counts()`` method

In [176]:
clean['PROZD'].value_counts() #[:6]

92    5
72    5
90    4
78    3
91    3
88    3
87    3
86    3
85    3
84    3
77    2
63    2
66    2
67    2
74    2
69    2
62    2
60    1
47    1
50    1
52    1
53    1
55    1
46    1
42    1
57    1
58    1
36    1
98    1
61    1
64    1
65    1
68    1
95    1
70    1
71    1
80    1
82    1
83    1
94    1
30    1
Name: PROZD, dtype: int64

In [177]:
type(clean['PROZD'].value_counts())

pandas.core.series.Series

We can use the values as keys to their counts

In [178]:
x  = clean['PROZD'].value_counts()

Example: to get the frequency (count) of the value `90`, we may use the value `90` as key to index the series and get the frequency, i.e. `x[63]`

In [182]:
x[90]

4

sanity check: the sum of the frequencies (counts) shoul be the same as the total number of values 

In [184]:
sum(x.values)

73

#### Useful tip: Comparing string values
Sometimes you need to convert string values to upper/lower case if letter case (capital/small) doesn't matter (i.e. case-insensitive comparison). strings has `.upper()` and `.lower()` to convert them

In [186]:
clean['ID'].str.lower().value_counts() #[:6]

kb     1
e2     1
e1     1
s1     1
goe    1
mar    1
sa3    1
w2     1
ms     1
v2     1
san    1
bi2    1
pb     1
tb     1
sz     1
fli    1
va     1
we     1
g2     1
bo     1
wd     1
bi1    1
r1     1
bn     1
mr     1
sa1    1
s2     1
wl     1
z      1
ey     1
      ..
r      1
cl     1
sa2    1
t1     1
li     1
wol    1
arn    1
g11    1
c      1
roe    1
hey    1
ml     1
gue    1
so     1
reh    1
wst    1
gra    1
s5     1
lue    1
kc     1
g5     1
hu     1
sm     1
ga1    1
wn     1
g6     1
v1     1
nie    1
d      1
be     1
Name: ID, Length: 73, dtype: int64

#### <font color =  #FF5733 > Q5: to find unique values in a column using ``unique()`` method

To find unique values in a column, you may use `.unique()` on that column

In [187]:
clean['ID'].unique()

array(['arn', 'be', 'bi1', 'bi2', 'bie', 'bn', 'bo', 'by', 'c', 'cl', 'd',
       'e1', 'e2', 'ey', 'fli', 'g10', 'g11', 'g2', 'g4', 'g5', 'g6',
       'ga1', 'goe', 'gra', 'gro', 'gue', 'hey', 'hu', 'hx', 'ka', 'kb',
       'kc', 'li', 'lue', 'mar', 'ml', 'mr', 'ms', 'n', 'nie', 'pb', 'r',
       'r1', 'r3', 'reh', 'roe', 's1', 's2', 's4', 's5', 'sa1', 'sa2',
       'sa3', 'san', 'sm', 'so', 'sz', 't1', 't2', 'tb', 'v1', 'v2', 'va',
       'w1', 'w2', 'wd', 'we', 'wl', 'wn', 'woe', 'wol', 'wst', 'z'],
      dtype=object)

In [188]:
clean['ID'].str.upper().unique()

array(['ARN', 'BE', 'BI1', 'BI2', 'BIE', 'BN', 'BO', 'BY', 'C', 'CL', 'D',
       'E1', 'E2', 'EY', 'FLI', 'G10', 'G11', 'G2', 'G4', 'G5', 'G6',
       'GA1', 'GOE', 'GRA', 'GRO', 'GUE', 'HEY', 'HU', 'HX', 'KA', 'KB',
       'KC', 'LI', 'LUE', 'MAR', 'ML', 'MR', 'MS', 'N', 'NIE', 'PB', 'R',
       'R1', 'R3', 'REH', 'ROE', 'S1', 'S2', 'S4', 'S5', 'SA1', 'SA2',
       'SA3', 'SAN', 'SM', 'SO', 'SZ', 'T1', 'T2', 'TB', 'V1', 'V2', 'VA',
       'W1', 'W2', 'WD', 'WE', 'WL', 'WN', 'WOE', 'WOL', 'WST', 'Z'],
      dtype=object)

In [189]:
clean['PROZD'].unique()

array([30, 72, 68, 42, 78, 69, 57, 63, 60, 67, 86, 71, 46, 84, 85, 87, 82,
       98, 92, 55, 58, 74, 91, 94, 95, 90, 77, 66, 62, 36, 65, 64, 88, 52,
       83, 50, 80, 53, 47, 70, 61], dtype=int64)

our frequency(count) series above has a unique index (key) values

In [192]:
len(x)

41

In [194]:
x.keys()

Int64Index([92, 72, 90, 78, 91, 88, 87, 86, 85, 84, 77, 63, 66, 67, 74, 69, 62,
            60, 47, 50, 52, 53, 55, 46, 42, 57, 58, 36, 98, 61, 64, 65, 68, 95,
            70, 71, 80, 82, 83, 94, 30],
           dtype='int64')

In [195]:
len(x.keys())

41

In [191]:
len(clean['PROZD'].unique())

41

## <font color =  #FF5733 > Learning activity

Write the code to do each of the following:

* Q1. list rows with `PSF` values between 25 and 40 
* Q2. Calculate the average of the `WDI` column of the rows listed in Q1
* Q3. From Q1 results, list all rows with `WDI` value greater than the average calculated in Q2


##### sample output included
* Q1

,ID,LBI,RTI,WDI,FLA,PSF,FSF,ZDF1,PROZD
3,bi2,1.08,43.7,2.40,105.0,32.6,5.8,10.7,42
10,d,1.23,27.6,2.83,121.0,27.9,6.7,31.7,67
12,e2,1.20,27.7,3.40,108.0,37.4,9.9,39.9,71
27,hu,1.41,23.8,3.20,118.0,25.5,15.5,26.8,72
34,mar,1.02,33.4,2.46,123.0,37.5,18.8,18.8,66
35,ml,1.04,29.4,3.10,120.0,28.6,12.7,15.9,62
37,ms,1.08,33.6,3.60,128.0,35.7,0.0,14.3,36
45,roe,1.26,27.5,2.41,121.0,31.7,20.2,24.0,62
60,v1,1.04,30.2,2.60,116.0,29.2,2.5,19.2,67
65,wd,1.16,36.0,3.00,127.0,36.4,10.2,9.4,53


* Q2

In [244]:
avg

2.891666666666667

* Q3

,ID,LBI,RTI,WDI,FLA,PSF,FSF,ZDF1,PROZD
12,e2,1.20,27.7,3.4,108.0,37.4,9.9,39.9,71
27,hu,1.41,23.8,3.2,118.0,25.5,15.5,26.8,72
35,ml,1.04,29.4,3.1,120.0,28.6,12.7,15.9,62
37,ms,1.08,33.6,3.6,128.0,35.7,0.0,14.3,36
65,wd,1.16,36.0,3.0,127.0,36.4,10.2,9.4,53
70,wol,1.24,27.2,3.0,120.0,29.1,13.8,34.4,70


### Next:
   * removing rows/columns using `.dropna()` with `how` or `thresh` parameters,
   * Replacing Null values with a substitue value (e.g., average) 
   * Extract descriptive statistics from the data
   * Normalize the data values
   * plot and read the correleation matrix of the data
   * plot and read the scatter matrix of the data